# Titanic competition - GaussianNB

In this approach I will try to reach great score using simple Mixed GaussianNB algorithm.

`var_smoothing` parameter is adjusted using Optuna

### Imports & settings

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split

import optuna

# MAX_DEPTH
MIN_MD = 2
MAX_MD = 3000

# MIN_SAMPLES_SPLIT
MIN_SS = 2
MAX_SS = 100

# MIN_SAMPLES_LEAF
MIN_SL = 1
MAX_SL = 150

# min_weight_fraction_leaf
MIN_WFL = 0
MAX_WFL = 0.4

# max_leaf_nodes
MIN_LN = 2
MAX_LN = 8000

#CCP
MIN_CCP = 0
MAX_CCP = 0.5

N_TRIALS = 20000
TIMEOUT = 60 * 60 * 6

DROP_LIST = ["PassengerId", "Name"]


### Get train dataset

In [2]:
titanic = pd.read_csv("./data/train.csv")
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Dataset INFO

In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### Drop columns
Which are difficult to process

In [4]:
titanic = titanic.drop(DROP_LIST, axis=1)
titanic

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


### Prepare data for algorithm
* In `Cabin` leave only firs letter (which means sector in which cabin is placed)
* Adjust labels for `Sex`, `Embarked` and `Cabin` using LabelEncoder
* Fill NaN values in `Age` with mean of this column

In [5]:
def preprocessing(data):
    data["Cabin"] = data.apply(
        lambda row: ord(str(row["Cabin"])[0]) - ord('A') if str(row["Cabin"])[0] != "n" else np.nan, axis=1)

    import re
    data["Ticket"] = data.apply(lambda row: re.sub("[^0-9]", "", row["Ticket"]), axis=1)
    data["Ticket_class"] = data.apply(lambda row: int(str(row["Ticket"])[0]) if len(str(row["Ticket"])) > 1 else np.nan,
                                      axis=1)
    data["Ticket_class"].fillna(data["Ticket_class"].mean(), inplace=True)
    data.drop(["Ticket"], axis=1, inplace=True)

    le = LabelEncoder()
    data["Sex"] = le.fit_transform(data["Sex"])
    data["Embarked"] = le.fit_transform(data["Embarked"])

    data["Age"].fillna(data["Age"].mean(), inplace=True)
    data["Cabin"].fillna(data["Cabin"].mean(), inplace=True)
    data.fillna(data.mean(), inplace=True)

    return data

In [6]:
preprocessing(titanic)
titanic

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_class
0,0,3,1,22.000000,1,0,7.2500,2.45098,2,5.0
1,1,1,0,38.000000,1,0,71.2833,2.00000,0,1.0
2,1,3,0,26.000000,0,0,7.9250,2.45098,2,2.0
3,1,1,0,35.000000,1,0,53.1000,2.00000,2,1.0
4,0,3,1,35.000000,0,0,8.0500,2.45098,2,3.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,2.45098,2,2.0
887,1,1,0,19.000000,0,0,30.0000,1.00000,2,1.0
888,0,3,0,29.699118,1,2,23.4500,2.45098,2,6.0
889,1,1,1,26.000000,0,0,30.0000,2.00000,0,1.0


### Split columns
to create datasets to predict and validate predictions.

`titanic_x` has to be casted to numpy array because of "categorical-feature" parameter of MixedNB.

In [7]:
titanic_x = titanic.drop(["Survived"], axis=1)  #.to_numpy()
titanic_y = titanic["Survived"]
titanic_x

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_class
0,3,1,22.000000,1,0,7.2500,2.45098,2,5.0
1,1,0,38.000000,1,0,71.2833,2.00000,0,1.0
2,3,0,26.000000,0,0,7.9250,2.45098,2,2.0
3,1,0,35.000000,1,0,53.1000,2.00000,2,1.0
4,3,1,35.000000,0,0,8.0500,2.45098,2,3.0
...,...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,2.45098,2,2.0
887,1,0,19.000000,0,0,30.0000,1.00000,2,1.0
888,3,0,29.699118,1,2,23.4500,2.45098,2,6.0
889,1,1,26.000000,0,0,30.0000,2.00000,0,1.0


Categorical columns are:

In [8]:
model = DecisionTreeClassifier(random_state=0)
acc = []

kf = KFold(n_splits=500, shuffle=True, random_state=1)
for train_index, test_index in kf.split(titanic_x):
    X_train, X_test = titanic_x.iloc[train_index, :], titanic_x.iloc[test_index, :]
    y_train, y_test = titanic_y.iloc[train_index], titanic_y.iloc[test_index]

    model.fit(X_train, y_train)
    acc.append(model.score(X_test, y_test))

print(np.mean(acc))

0.795


In [9]:
def objective(trial):
    kf = KFold(n_splits=10, shuffle=True, random_state=8)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", MIN_MD, MAX_MD)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", MIN_SL, MAX_SL)
    min_samples_split = trial.suggest_int("min_samples_split", MIN_SS, MAX_SS)
    min_weight_fraction_leaf = trial.suggest_float("min_weight_fraction_leaf", MIN_WFL, MAX_WFL)
    max_leaf_nodes = trial.suggest_int("max_leaf_nodes", MIN_LN, MAX_LN)
    ccp_alpha = trial.suggest_float("ccp_alpha", MIN_CCP, MAX_CCP)

    model = DecisionTreeClassifier(criterion=criterion,
                                   max_depth=max_depth,
                                   min_samples_leaf=min_samples_leaf,
                                   min_samples_split=min_samples_split,
                                   min_weight_fraction_leaf=min_weight_fraction_leaf,
                                   max_leaf_nodes=max_leaf_nodes,
                                   ccp_alpha=ccp_alpha,
                                   random_state=10)
    acc = []

    for train_index, test_index in kf.split(titanic_x):
        X_train, X_test = titanic_x.iloc[train_index, :], titanic_x.iloc[test_index, :]
        y_train, y_test = titanic_y.iloc[train_index], titanic_y.iloc[test_index]

        model.fit(X_train, y_train)
        acc.append(model.score(X_test, y_test))
    return np.mean(acc)

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=TIMEOUT, n_trials=N_TRIALS, n_jobs=-1)
print(f"BEST TRIAL: {study.best_trial}")

# optuna.visualization.plot_slice(study).show()

[I 2022-05-18 17:11:51,244] A new study created in memory with name: no-name-8a7677ce-3c51-484f-9dc6-c78814b3805d
C:\ProgramData\Anaconda3\envs\podstawy_sztucznej_inteligencji\lib\site-packages\optuna\study\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-05-18 17:11:52,029] Trial 2 finished with value: 0.6161298377028714 and parameters: {'criterion': 'gini', 'max_depth': 2892, 'min_samples_leaf': 41, 'min_samples_split': 91, 'min_weight_fraction_leaf': 0.26829166372710467, 'max_leaf_nodes': 3424, 'ccp_alpha': 0.3967759097603796}. Best is trial 2 with value: 0.6161298377028714.
[I 2022-05-18 17:11:52,079] Trial 1 finished with value: 0.6161298377028714 and parameters: {'criterion': 'entropy', 'max_depth': 227, 'min_samples_leaf': 94, 'min_samples_split': 20, 'min_weight_fraction_leaf': 0.28352131729664726, 'max_leaf_nodes': 3015, 'ccp_a

[I 2022-05-18 17:11:53,211] Trial 28 finished with value: 0.776716604244694 and parameters: {'criterion': 'entropy', 'max_depth': 1039, 'min_samples_leaf': 114, 'min_samples_split': 48, 'min_weight_fraction_leaf': 0.1637753409431958, 'max_leaf_nodes': 4941, 'ccp_alpha': 0.030769449920142536}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:53,238] Trial 29 finished with value: 0.7643570536828964 and parameters: {'criterion': 'entropy', 'max_depth': 791, 'min_samples_leaf': 114, 'min_samples_split': 46, 'min_weight_fraction_leaf': 0.14549878606939437, 'max_leaf_nodes': 5318, 'ccp_alpha': 0.06414763720744994}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:53,249] Trial 31 finished with value: 0.7643570536828964 and parameters: {'criterion': 'entropy', 'max_depth': 867, 'min_samples_leaf': 119, 'min_samples_split': 47, 'min_weight_fraction_leaf': 0.14837828348253715, 'max_leaf_nodes': 5295, 'ccp_alpha': 0.0006663853121124733}. Best is trial 0 with

[I 2022-05-18 17:11:54,851] Trial 42 finished with value: 0.7868289637952559 and parameters: {'criterion': 'entropy', 'max_depth': 588, 'min_samples_leaf': 140, 'min_samples_split': 39, 'min_weight_fraction_leaf': 0.2215411530047281, 'max_leaf_nodes': 2913, 'ccp_alpha': 0.10822168266428485}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:54,901] Trial 52 finished with value: 0.7868289637952559 and parameters: {'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 138, 'min_samples_split': 19, 'min_weight_fraction_leaf': 0.2384101669644355, 'max_leaf_nodes': 2916, 'ccp_alpha': 0.12539080961876586}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:54,937] Trial 49 finished with value: 0.7868289637952559 and parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 137, 'min_samples_split': 38, 'min_weight_fraction_leaf': 0.10877266232437585, 'max_leaf_nodes': 3460, 'ccp_alpha': 0.11431749732676738}. Best is trial 0 with value

[I 2022-05-18 17:11:56,309] Trial 71 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1194, 'min_samples_leaf': 126, 'min_samples_split': 54, 'min_weight_fraction_leaf': 0.17980300793175094, 'max_leaf_nodes': 6324, 'ccp_alpha': 0.05087886670123945}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:56,325] Trial 72 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1233, 'min_samples_leaf': 65, 'min_samples_split': 55, 'min_weight_fraction_leaf': 0.18154854044286306, 'max_leaf_nodes': 6489, 'ccp_alpha': 0.057873898535377874}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:56,424] Trial 73 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1218, 'min_samples_leaf': 64, 'min_samples_split': 43, 'min_weight_fraction_leaf': 0.1841039078876721, 'max_leaf_nodes': 6495, 'ccp_alpha': 0.05943605949020455}. Best is trial 0 with value: 0.

[I 2022-05-18 17:11:57,702] Trial 90 finished with value: 0.7868289637952559 and parameters: {'criterion': 'entropy', 'max_depth': 293, 'min_samples_leaf': 137, 'min_samples_split': 28, 'min_weight_fraction_leaf': 0.3141310921382015, 'max_leaf_nodes': 3684, 'ccp_alpha': 0.1359836027439188}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:57,711] Trial 91 finished with value: 0.7868289637952559 and parameters: {'criterion': 'entropy', 'max_depth': 335, 'min_samples_leaf': 137, 'min_samples_split': 28, 'min_weight_fraction_leaf': 0.305881184118449, 'max_leaf_nodes': 3695, 'ccp_alpha': 0.13544937894472128}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:11:57,713] Trial 98 finished with value: 0.7868289637952559 and parameters: {'criterion': 'entropy', 'max_depth': 141, 'min_samples_leaf': 102, 'min_samples_split': 22, 'min_weight_fraction_leaf': 0.12468361322301758, 'max_leaf_nodes': 3473, 'ccp_alpha': 0.13114920881359182}. Best is trial 0 with value

...

[I 2022-05-18 17:20:43,654] Trial 3678 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1467, 'min_samples_leaf': 1, 'min_samples_split': 71, 'min_weight_fraction_leaf': 0.17492954183929874, 'max_leaf_nodes': 6585, 'ccp_alpha': 0.1127375138677866}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:20:44,062] Trial 3679 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1477, 'min_samples_leaf': 1, 'min_samples_split': 71, 'min_weight_fraction_leaf': 0.17445194756331817, 'max_leaf_nodes': 6574, 'ccp_alpha': 0.1121625953758752}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:20:44,405] Trial 3675 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1452, 'min_samples_leaf': 1, 'min_samples_split': 71, 'min_weight_fraction_leaf': 0.17346231788335154, 'max_leaf_nodes': 6593, 'ccp_alpha': 0.1119145977071321}. Best is trial 0 with value: 0.7

[I 2022-05-18 17:20:48,848] Trial 3703 finished with value: 0.6161298377028714 and parameters: {'criterion': 'gini', 'max_depth': 1307, 'min_samples_leaf': 8, 'min_samples_split': 68, 'min_weight_fraction_leaf': 0.18336651105573928, 'max_leaf_nodes': 6339, 'ccp_alpha': 0.32713155577769293}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:20:48,854] Trial 3702 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1351, 'min_samples_leaf': 8, 'min_samples_split': 68, 'min_weight_fraction_leaf': 0.18361000211372694, 'max_leaf_nodes': 6370, 'ccp_alpha': 0.1197979995745253}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:20:49,049] Trial 3696 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1110, 'min_samples_leaf': 8, 'min_samples_split': 68, 'min_weight_fraction_leaf': 0.18416070584204278, 'max_leaf_nodes': 6394, 'ccp_alpha': 0.1203799734741452}. Best is trial 0 with value: 0.

[I 2022-05-18 17:20:54,988] Trial 3727 finished with value: 0.7385143570536828 and parameters: {'criterion': 'gini', 'max_depth': 1256, 'min_samples_leaf': 10, 'min_samples_split': 66, 'min_weight_fraction_leaf': 0.18567724983877038, 'max_leaf_nodes': 6008, 'ccp_alpha': 0.1321897826397377}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:20:55,026] Trial 3726 finished with value: 0.7385143570536828 and parameters: {'criterion': 'gini', 'max_depth': 1215, 'min_samples_leaf': 10, 'min_samples_split': 66, 'min_weight_fraction_leaf': 0.18587484184391354, 'max_leaf_nodes': 6041, 'ccp_alpha': 0.13250634089679375}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:20:55,027] Trial 3728 finished with value: 0.7598626716604244 and parameters: {'criterion': 'gini', 'max_depth': 1233, 'min_samples_leaf': 9, 'min_samples_split': 67, 'min_weight_fraction_leaf': 0.18700267304296306, 'max_leaf_nodes': 6003, 'ccp_alpha': 0.130349356937889}. Best is trial 0 with value: 0

[I 2022-05-18 17:21:00,522] Trial 3749 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1325, 'min_samples_leaf': 11, 'min_samples_split': 61, 'min_weight_fraction_leaf': 0.19208957780321884, 'max_leaf_nodes': 6161, 'ccp_alpha': 0.12902356615171082}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:21:00,798] Trial 3745 finished with value: 0.7385143570536828 and parameters: {'criterion': 'gini', 'max_depth': 1312, 'min_samples_leaf': 11, 'min_samples_split': 61, 'min_weight_fraction_leaf': 0.19724282805030705, 'max_leaf_nodes': 6176, 'ccp_alpha': 0.13539304928295023}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:21:01,502] Trial 3750 finished with value: 0.7868289637952559 and parameters: {'criterion': 'gini', 'max_depth': 1066, 'min_samples_leaf': 10, 'min_samples_split': 61, 'min_weight_fraction_leaf': 0.1971714585727504, 'max_leaf_nodes': 6143, 'ccp_alpha': 0.12669833452184415}. Best is trial 0 with value

[I 2022-05-18 17:21:08,131] Trial 3780 finished with value: 0.6407615480649189 and parameters: {'criterion': 'gini', 'max_depth': 1077, 'min_samples_leaf': 2, 'min_samples_split': 61, 'min_weight_fraction_leaf': 0.20065914435481946, 'max_leaf_nodes': 6181, 'ccp_alpha': 0.14402529973000017}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:21:08,288] Trial 3784 finished with value: 0.7385143570536828 and parameters: {'criterion': 'gini', 'max_depth': 1144, 'min_samples_leaf': 2, 'min_samples_split': 60, 'min_weight_fraction_leaf': 0.1997974885384792, 'max_leaf_nodes': 6107, 'ccp_alpha': 0.13514560052392166}. Best is trial 0 with value: 0.7868289637952559.
[I 2022-05-18 17:21:08,314] Trial 3776 finished with value: 0.7868289637952559 and parameters: {'criterion': 'entropy', 'max_depth': 606, 'min_samples_leaf': 94, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.3174899353926734, 'max_leaf_nodes': 6165, 'ccp_alpha': 0.14162238670972474}. Best is trial 0 with value: 

In [ ]:
optuna.visualization.plot_slice(study).show()

## Generate test output

In [ ]:
titanic_test = pd.read_csv("./data/test.csv")
ids = titanic_test["PassengerId"]
print(f"SHAPE: {titanic_test.shape}")
titanic_test = titanic_test.drop(DROP_LIST, axis=1)
preprocessing(titanic_test)
titanic_test

In [ ]:
model = DecisionTreeClassifier(criterion=study.best_trial.params["criterion"],
                               max_depth=study.best_trial.params["max_depth"],
                               min_samples_leaf=study.best_trial.params["min_samples_leaf"],
                               min_samples_split=study.best_trial.params["min_samples_split"],
                               min_weight_fraction_leaf=study.best_trial.params["min_weight_fraction_leaf"],
                               max_leaf_nodes=study.best_trial.params["max_leaf_nodes"],
                               ccp_alpha=study.best_trial.params["ccp_alpha"],
                               random_state=10)
model.fit(titanic_x, titanic_y)
survived = model.predict(titanic_test)
survived = pd.DataFrame(survived, columns=["Survived"])
survived["PassengerId"] = ids
survived.to_csv("./results.csv", index=False)